In [1]:
# !pip install pypdf pandas

In [14]:
year = 2019
month = 1

assert 1 <= month <= 12
assert 2019 <= year <= 2023

url = f"https://www.vedur.is/media/vedurstofan/utgafa/hlidarefni/climat/climat-Reykjavik-{month}-{year}.pdf"
url = url.format(month=month, year=year)

# Download the pdf
import urllib.request

pdf_path = f"reykjavik-{month}-{year}.pdf"
urllib.request.urlretrieve(url, pdf_path)


('reykjavik-1-2019.pdf', <http.client.HTTPMessage at 0x7f09eb21abe0>)

In [15]:
from pypdf import PdfReader, PdfWriter


def crop_pdf(input_path, output_path, x1, y1, x2, y2):
    with open(input_path, "rb") as file:
        input_pdf = PdfReader(file)
        num_pages = len(input_pdf.pages)

        output_pdf = PdfWriter()

        for page_number in range(num_pages):
            page = input_pdf.pages[page_number]
            page.cropbox.lower_left = (x1, y1)
            page.cropbox.upper_right = (x2, y2)
            output_pdf.add_page(page)

        with open(output_path, "wb") as output_file:
            output_pdf.write(output_file)


crop_pdf(pdf_path, pdf_path, 25, 178, 375, 770)

In [16]:
# importing required modules
from pypdf import PdfReader
import pandas as pd
import re

# creating a pdf file object

pdf_file = open(pdf_path, "rb")

# creating a pdf reader object
reader = PdfReader(pdf_file)

# creating a page object
page = reader.pages[0]  # Access the first page

table_content = page.extract_text(0, 90)
table_content = table_content.split("\n")

table_content = table_content[:-27]
table_content = [x.strip() for x in table_content]


amount_of_columns = 12
amount_of_rows = 31
index_of_end_of_table = []
for i in range(amount_of_columns):
    index = amount_of_rows - i + (amount_of_rows * i)
    index_of_end_of_table.append(index)


data = []

print(index_of_end_of_table)

for i, split_index in enumerate(index_of_end_of_table):
    if i == 0:
        data.append(table_content[:split_index])
    else:
        data.append(table_content[index_of_end_of_table[i - 1] : split_index])


# fix last cell in each row
for i, row in enumerate(data):
    last_cell = row[-1]

    if "−" in last_cell:
        last_cell = last_cell.replace("−", " -")
        last_cell = last_cell.split(" ")
        last_cell = [x for x in last_cell if x != ""]

        row[-1] = last_cell[0]

        if i + 1 < len(data):
            data[i + 1].insert(0, last_cell[1])

    if " " in last_cell:
        last_cell = last_cell.split(" ")

        row[-1] = last_cell[0]

        if i + 1 < len(data):
            data[i + 1].insert(0, last_cell[1])

    if "." in last_cell and len(last_cell) > 4:
        # Match the last decimal place and add a space after it
        modified_string = re.sub(r"(\.\d+)(?!\.)", r"\1 ", last_cell)
        modified_string = modified_string.split(" ")
        modified_string = [x for x in modified_string if x != ""]

        row[-1] = modified_string[0]

        if i + 1 < len(data):
            data[i + 1].insert(0, modified_string[1])

    # if last cell does not match amount of decimals
    if "." in last_cell and len(last_cell.split(".")[-1]) > 1:
        # split after last decimal place
        modified_string = last_cell.split(".")
        modified_string = modified_string[0] + "." + modified_string[1][0]

        new_cell = last_cell.replace(modified_string, "")

        row[-1] = modified_string

        if i + 1 < len(data):
            data[i + 1].insert(0, new_cell)



    # if rain type column
    if (
        i == 5
        and last_cell != ""
        and last_cell != "sn"
        and last_cell != "sl"
        and last_cell != "ri"
    ):
        row[-1] = ""

        if i + 1 < len(data):
            data[i + 1].insert(0, last_cell)

    # if snow type column has too large value
    if i == 6 and len(last_cell) > 2:
        corrected_value = last_cell[:2]
        row[-1] = corrected_value

        if i + 1 < len(data):
            data[i + 1].insert(0, last_cell[2:])

    # if range of snow cover column has too large value
    if i == 7 and len(last_cell) > 1:
        corrected_value = last_cell[:1]
        row[-1] = corrected_value

        if i + 1 < len(data):
            data[i + 1].insert(0, last_cell[1:])

    # replace "−" with "-" in all cells
    for j, cell in enumerate(row):
        if "−" in cell:
            row[j] = cell.replace("−", "-")
            
    print(i, len(row), row)


header = [
    "day",
    "avg temperature",
    "max temp",
    "min temp",
    "rain",
    "rain type",
    "snow depth",
    "snow type",
    "sun hours",
    "avg wind",
    "max wind",
    "gust",
]

# rotate data
data = list(map(list, zip(*data)))

try:
    df = pd.DataFrame(data, columns=header)
    # print(data)

    csv_path = pdf_path.split("/")[-1].replace(".pdf", ".csv")
    df.to_csv("../data/climat/" + csv_path, index=False)
except:
    print("Error in data, could not convert to csv")

[31, 61, 91, 121, 151, 181, 211, 241, 271, 301, 331, 361]
0 31 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
1 31 ['-0.2', '7.7', '8.0', '8.8', '4.0', '3.0', '3.0', '3.7', '7.9', '3.3', '5.7', '1.1', '-1.0', '-0.6', '-0.2', '-4.8', '-0.2', '3.3', '1.4', '-0.1', '-1.7', '-2.6', '-3.8', '-0.9', '-1.1', '-5.0', '-3.5', '-4.3', '-5.9', '-6.3', '-4.7']
2 31 ['2.4', '8.2', '9.9', '9.6', '9.0', '5.0', '5.1', '5.4', '9.1', '8.6', '7.9', '5.0', '1.3', '2.5', '2.5', '0.3', '0.3', '4.3', '5.2', '0.8', '4.0', '0.7', '-0.6', '0.6', '1.4', '0.8', '-1.2', '-0.9', '-2.1', '-3.4', '-2.0']
3 31 ['-7.2', '1.5', '6.1', '5.7', '0.7', '0.3', '2.2', '-0.2', '4.8', '2.4', '2.5', '0.3', '-3.0', '-3.6', '-1.4', '-7.4', '-6.4', '0.2', '-0.7', '-2.1', '-3.1', '-4.5', '-6.0', '-7.1', '-4.4', '-8.0', '-9.0', '-6.6', '-8.4', '-9.1', '-9.3']
4 31 ['0.1', '1.1', '11.9', '0.8', '6.1', '4.0